In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import pathlib
import swifter
from functools import reduce

from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, MultiPoint

from geovoronoi import voronoi_regions_from_coords
crs4326 = {'init': 'epsg:4326'}


In [2]:
DIR_ROOT = os.path.join(pathlib.Path().absolute(), '../..' )

positions = pd.read_csv(DIR_ROOT + '/data/04_itinerary/positions.csv')
positions['geometry'] = positions['geometry'].apply(wkt.loads)
positions = gpd.GeoDataFrame(positions, crs=crs4326).set_geometry('geometry')

localities = pd.read_csv(DIR_ROOT + '/data/02_processed/localities.csv')
localities['geometry'] = localities['geometry'].apply(wkt.loads)
localities = gpd.GeoDataFrame(localities, crs=crs4326).set_geometry('geometry')

bbox = localities.buffer(0.01).total_bounds

<ipython-input-2-001be3eb2419>:11: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bbox = localities.buffer(0.01).total_bounds


In [3]:
localities = localities[~(localities.geometry.is_empty | localities.geometry.isna())]
localities = localities[~(localities.region2.isna())].reset_index()
localities

<ipython-input-3-098b9e772dc9>:1: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  localities = localities[~(localities.geometry.is_empty | localities.geometry.isna())]


,index,Unnamed: 0,id,name,no_alle,no_activity,no_reise,no_bi,no_gg,x,y,prazision,noten,region1,region2,region3,geometry
0,0,0,1,Absdorf,1,0,0,0,1,48.40,15.98,1.0,NaN,Niederösterreich,Nordostniederösterreich,Nordostniederösterreich,POINT (15.98000 48.40000)
1,1,1,2,Admont,6,0,0,6,0,47.58,14.47,1.0,NaN,Steiermark,Nordsteiermark,Nordsteiermark,POINT (14.47000 47.58000)
2,2,2,3,Aisthofen in Riedmark,1,0,0,0,1,48.26,14.59,3.0,NaN,Niederösterreich,Nordwestniederösterreich,Nordwestniederösterreich,POINT (14.59000 48.26000)
3,3,3,4,Aldersbach,1,0,0,1,0,48.59,13.09,1.0,NaN,Bayern,Bayern,Bayern,POINT (13.09000 48.59000)
4,4,4,5,Alland,1,0,0,0,1,48.06,16.08,1.0,NaN,Niederösterreich,Südostniederösterreich,Südostniederösterreich,POINT (16.08000 48.06000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,664,664,665,Zlíchov,2,0,0,0,2,50.05,14.41,1.0,NaN,Böhmen,Mittelböhmen,Mittelböhmen,POINT (14.41000 50.05000)
600,666,666,667,Znojmo,59,27,27,3,2,48.86,16.05,1.0,NaN,Mähren,Znaimer Teil,Znaimer Teil,POINT (16.05000 48.86000)
601,667,667,668,Zvíkov,5,2,2,1,0,49.44,14.19,1.0,NaN,Böhmen,Südböhmen,Südböhmen,POINT (14.19000 49.44000)
602,668,668,669,Zwerndorf,1,0,0,0,1,48.34,16.84,1.0,NaN,Niederösterreich,Nordostniederösterreich,Nordostniederösterreich,POINT (16.84000 48.34000)


In [4]:
bounds = Polygon([
    [bbox[0], bbox[1]], 
    [bbox[0], bbox[3]], 
    [bbox[2], bbox[3]], 
    [bbox[2], bbox[1]]])

polys, poly_ids = voronoi_regions_from_coords(localities['geometry'], bounds)

In [10]:
regions_positions_1 = {}
regions_positions_2 = {}
regions_positions_3 = {}
regions_positions_4 = {}
regions_positions_5 = {}
regions_positions_all = {}

for region_name in localities['region2'].unique():
    regions_positions_1[region_name] = 0
    regions_positions_2[region_name] = 0
    regions_positions_3[region_name] = 0
    regions_positions_4[region_name] = 0
    regions_positions_5[region_name] = 0
    regions_positions_all[region_name] = 0


In [11]:
region_positionsvoronois = []

for pi, poly in enumerate(polys.items()):
    poly_geo = poly[1]

    if pi < 1000:
        #poly_localities = localities[localities['id'].isin(poly_ids)]
        for poly_id in poly_ids[pi]:
            poly_locality = localities.loc[poly_id]
            region_name = poly_locality['region2']
            region_positions = positions[positions.intersects(poly_geo)]

            p_all = len(region_positions.index)
            p_1 = len(region_positions[region_positions['period'] == 1].index)
            p_2 = len(region_positions[region_positions['period'] == 2].index)
            p_3 = len(region_positions[region_positions['period'] == 3].index)
            p_4 = len(region_positions[region_positions['period'] == 4].index)
            p_5 = len(region_positions[region_positions['period'] == 5].index)

            regions_positions_all[region_name] += p_all
            regions_positions_1[region_name] += p_1
            regions_positions_2[region_name] += p_2
            regions_positions_3[region_name] += p_3
            regions_positions_4[region_name] += p_4
            regions_positions_5[region_name] += p_5

            print(pi, int(pi / len(polys) * 100), len(region_positions.index), poly_locality['name'], region_name)
            if p_all != p_1 + p_2 + p_3 + p_4 + p_5:
                print('problem')
                
                print(p_all, p_1, p_2, p_3, p_4, p_5)
                print(region_positions['period'] == 2)

            #print(region_positions)

            # print(len(region_positions.index))

            # print(len(region_positions[region_positions['period'] == 1].index))
            # print(len(region_positions[region_positions['period'] == 2].index))
            # print(len(region_positions[region_positions['period'] == 3].index))
            # print(len(region_positions[region_positions['period'] == 4].index))
            # print(len(region_positions[region_positions['period'] == 5].index))

            #print(int(pi / len(polys) * 100), len(region_positions.index), poly_locality['name'], region_name)
            
            #print(poly_locality)
            #voronois


        # for poly_locality in poly_localities.iterrows():
        #     #print(poly_locality[1])
        #     region_name = poly_locality[1]['region2']
        #     region_positions = positions[positions.intersects(poly_geo)]
        #     regions_positions[region_name] += len(region_positions)


Salzburg
69 11 22 Kappling Oberösterreich
70 11 0 Puchheim Oberösterreich
71 11 0 Bad Ischl Oberösterreich
72 12 0 Rinchnach Bayern
73 12 0 Aldersbach Bayern
74 12 0 Vöcklabruck Oberösterreich
75 12 0 Mondsee Salzburg
76 12 282 Litomyšl Ostböhmen
77 12 0 Uherský Brod Südostmähren
78 13 0 Těšov Südostmähren
79 13 0 Vizovice Nordostmähren
80 13 0 Benešov Troppauer Teil
81 13 0 Hněvkovice Böhmischmährische Höhe
82 13 1097 Havlíčkův Brod Böhmischmährische Höhe
83 13 0 Marienthal Nordböhmen
84 14 14 Oberseifersdorf Nordböhmen
85 14 0 Frýdlant Nordböhmen
86 14 0 Ostritz Nordböhmen
86 14 0 Russdorf bei Ostritz Nordböhmen
87 14 163 Opole Schlesien
88 14 954 Wrocław Schlesien
89 15 228 Kalisz Pollen
90 15 142 Ujazd Troppauer Teil
91 15 0 Trzebnica Schlesien
92 15 58 Grobniki Troppauer Teil
93 15 385 Kouřim Ostböhmen
94 15 553 Poboří Ostböhmen
95 16 16 Ždánice Ostböhmen
95 16 16 Železná Mittelböhmen
96 16 115 Nymburk Ostböhmen
97 16 3536 Sadská Ostböhmen
98 16 42 Úherce Nordböhmen
99 16 3298 Pod

In [12]:
# test
print(regions_positions_1['Bayern'] + regions_positions_2['Bayern'] + regions_positions_3['Bayern'] + regions_positions_4['Bayern'] + regions_positions_5['Bayern'] == regions_positions_all['Bayern'])

True


In [13]:
df1 = pd.DataFrame(regions_positions_1.items(), columns=['region', 'p-1'])
df2 = pd.DataFrame(regions_positions_2.items(), columns=['region', 'p-2'])
df3 = pd.DataFrame(regions_positions_3.items(), columns=['region', 'p-3'])
df4 = pd.DataFrame(regions_positions_4.items(), columns=['region', 'p-4'])
df5 = pd.DataFrame(regions_positions_5.items(), columns=['region', 'p-5'])
dfa = pd.DataFrame(regions_positions_all.items(), columns=['region', 'p-all'])

dfs = [df1, df2, df3, df4, df5, dfa]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['region'],
                                            how='outer'), dfs)
df_merged.to_csv(DIR_ROOT + '/data/06_outputs/regions-time.csv', sep=",")


In [27]:
# create geometries for aggregated regions

def get_voronois_for_region(region_name):
    poly_by_region = []
    for pi, poly in enumerate(polys.items()):
        for poly_id in poly_ids[pi]:
            poly_locality = localities.loc[poly_id]
            poly_region_name = poly_locality['region2']
            
            if region_name == poly_region_name:
                poly_geo = poly[1]
                poly_by_region.append(poly_geo)
    return shapely.ops.cascaded_union(poly_by_region)


df_merged['geometry'] = df_merged.apply(
    lambda x: get_voronois_for_region(x['region']),
    axis=1
)

gpd.GeoDataFrame(df_merged).to_file(DIR_ROOT + '/data/06_outputs/regions-time-geometries.geojson', driver="GeoJSON")